In [252]:
import pandas as pd
import chardet
import json
import requests

filename = 'group_txt_test.csv'

Получение списка всех групп из файла и всех items в них:

In [262]:
def get_groups_from_file(filename):
    with open(filename, 'rb') as f:
        result = chardet.detect(f.read())
        groups_with_items = pd.read_csv(filename, sep=';', comment='#', encoding=result['encoding'])

    grouped = groups_with_items.groupby(['_JK_name', 'External_ID_user']).agg(list).reset_index()
    df_grouped = pd.DataFrame({
        'id': grouped['External_ID_user'], 
        'description': grouped['_JK_name'].map(str) + ' ' + grouped['External_ID_user'].map(str),
        'itemIds': grouped['External_id_item']
    })
    df_grouped.reset_index(drop=True, inplace=True)
    return df_grouped

Получение всех групп из API:

In [227]:
def get_groups_from_api():
    url = 'http://app00:8888/service-calltracking'+'/itemGroup/search'
    header = {'content-type': 'application/json'}

    payload = {'id': 21}
    r = requests.post(url, headers = header) #,data=json.dumps(payload))
    data = json.loads(r.text)
   
    df = pd.DataFrame(data['result']['items'])
    df_groups_api = pd.DataFrame({
        'groupId': df['groupId'],
        'id': df['userId'], 
        'description': df['description'],
        'itemIds': df['itemIds']})
    df_groups_api.reset_index(drop=True, inplace=True)
    return df_groups_api

**Сравнение списков групп:**
находим те группы, которые есть в файле, но отсутствуют в API

In [229]:
def get_new_groups(filename):
    file_groups = get_groups_from_file(filename)
    api_groups = get_groups_from_api().drop('groupId', 1)

    # ищем разные строки в файле и выгрузке из api
    # это работает, если в датафреймах нет дубликатов (их у нас не должно быть)
    diff = pd.concat([file_groups, api_groups]).drop_duplicates(['id', 'description'], keep=False)

    # оставляем только те строки из diff, которые есть в файле
    diff = diff[diff.isin(file_groups)].dropna()
    diff.reset_index(drop=True, inplace=True)
    return diff

Добавление новых групп и их items в API:
<br/>(на вход подается датафрейм с новыми группами, которых нет еще в api)

In [165]:
def add_new_groups(df_new_groups):
    url = 'http://app00:8888/service-calltracking'+'/itemGroup/createGroup'
    for i in range(0, len(df_new_groups)):
        data = {
                'userId': int(df_new_groups['id'][i]),
                'description': df_new_groups['description'][i],
                'itemIds': df_new_groups['itemIds'][i]
                    }
        r = requests.post(url, data = data)

Поиск отсутствующих items для существующих групп:

In [297]:
def get_new_items_for_existing_groups():
    df1 = get_groups_from_file(filename)
    df2 = get_groups_from_api().drop('groupId', 1)

    # в df актуальные items
    df = pd.concat([df1,df2], sort=True)
    df['itemIds_actual'] = df['itemIds']

    # для удаления дубликатов сравниваем список itemIds в string формате
    df['itemIds_actual_str'] = df['itemIds'].astype(str)
    df.drop_duplicates(['id','description', 'itemIds_actual_str'], keep=False)
    
    # удаляем дубликаты новых групп
    df = pd.concat([df, get_new_groups(filename)], sort=True).drop_duplicates(['id','description'], keep=False)
    df.reset_index(drop=True, inplace=True)

    # join таблиц по description, чтобы потом найти недостающие items для api
    df.set_index('description').join(df2.set_index('description'), lsuffix='_actual', rsuffix='_api', on='description')

    def diff(first, second):
        second = set(second)
        return [item for item in first if item not in second]

    # в этом столбце значения items, которые нужно добавить в api
    df['new_items'] = df.apply(lambda row: list(set(row.itemIds_actual) - set(row.itemIds)), axis=1)

    df = df.drop(['itemIds', 'itemIds_actual', 'itemIds_actual_str'], axis=1)
    
    # если пустой массив новых items, то исключаем строку
    #df = df[df['new_items'].map(lambda d: len(d)) > 0]
    return df


Получение groupId через userId и description:

In [300]:
def get_group_id_for_new_items():
    df_new_items = get_new_items_for_existing_groups()
    api_groups = get_groups_from_api()

    df_new_items_with_groupid = pd.merge(df_new_items, api_groups, on=['description', 'id'], how='left')
    return df_new_items_with_groupid

Добавление отсутствующих items для уже существующих групп в API:
<br/>(на вход подается df из метода get_group_id_for_new_items() )

In [299]:
def add_new_items_to_existing_groups(df_new_items_with_groupid):
    url = 'http://app00:8888/service-calltracking'+'/itemGroup/addItemToGroup'
    for i in range(0, len(df_new_items_with_groupid)):
        data = {
                'groupId': df_new_items_with_groupid['groupId'][i],
                'itemIds': df_new_items_with_groupid['new_items'][i]
                    }
        r = requests.post(url, data = data)